<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Audio_deepfake_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####  **Audio Deepfake Detection, Fake Calls, Spoofing, Fraud Calls and Voice Cloning Analysis for Defensice Forensics**

In [3]:
!pip install -q numpy librosa soundfile matplotlib IPython webrtcvad pydub noisereduce pyAudioAnalysis speechbrain langchain openai langgraph transformers vllm requests ipywidgets
!pip install -q audiomentations hmmlearn eyed3 langchain_community praat-parselmouth webrtcvad groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 9.0 MB/s eta 0:00:00


In [4]:
import os
import json
import asyncio
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import librosa, librosa.display, matplotlib.pyplot as plt
import IPython.display as ipd
from IPython.display import display, clear_output, HTML
import webrtcvad
from pydub import AudioSegment
import noisereduce as nr
from pyAudioAnalysis import audioSegmentation as aS, ShortTermFeatures
import parselmouth
import torch
from moviepy.editor import VideoFileClip

# Additional library for audio augmentations.
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

# SpeechBrain modules for inference and augmentation
from speechbrain.inference.speaker import SpeakerRecognition
try:
    from speechbrain.augment import AddNoise
except ImportError:
    AddNoise = None

# LangChain modules for LLM-based report generation
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import openai

# vLLM modules for multimodal inference
from transformers import AutoTokenizer
from vllm import LLM, EngineArgs, SamplingParams

# Groq SDK for audio processing (ensure pip install groq)
from groq import Groq

import ipywidgets as widgets

In [5]:
# ----- Data Model -----
class ForensicReport:
    def __init__(self, **kwargs):
        self.file = kwargs.get("file")
        self.verdict = kwargs.get("verdict")
        self.mean_score = kwargs.get("mean_score")
        self.confidence = kwargs.get("confidence")
        self.all_model_scores = kwargs.get("all_model_scores")
        self.all_anomalies = kwargs.get("all_anomalies")
        self.natural_summary = kwargs.get("natural_summary")
        self.asr_transcript = kwargs.get("asr_transcript")
        self.asr_lang = kwargs.get("asr_lang")
        self.speaker_identities = kwargs.get("speaker_identities")
        self.speaker_spoof_score = kwargs.get("speaker_spoof_score")
        self.noise_quality_score = kwargs.get("noise_quality_score")
        self.gender_distribution = kwargs.get("gender_distribution")
        self.additional_metrics = kwargs.get("additional_metrics", {})
        self.plots = kwargs.get("plots", {})
        self.timestamp = kwargs.get("timestamp")
        self.vllm_model_outputs = kwargs.get("vllm_model_outputs", {})
        self.analysis_text = kwargs.get("analysis_text", "")
        self.groq_transcription = kwargs.get("groq_transcription", "")
        self.groq_translation = kwargs.get("groq_translation", "")
        self.groq_tts_audio = kwargs.get("groq_tts_audio", "")

    def json(self, indent=2):
        return json.dumps(self.__dict__, indent=indent)

# ----- Global Async Executor -----
executor = ThreadPoolExecutor(max_workers=8)

In [6]:
# ----- Additional Optimization: Microbatching Helper -----
def microbatch_process(fn, data, batch_size):
    results = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        results.append(fn(batch))
    if isinstance(results[0], np.ndarray):
        return np.concatenate(results, axis=0)
    return results

# ----- Groq API Functions -----
def get_groq_client():
    api_key = os.environ.get("GROQ_API_KEY")
    if not api_key:
        raise ValueError("Please set the GROQ_API_KEY environment variable.")
    return Groq(api_key=api_key)

def groq_transcribe(audio_path, model="whisper-large-v3-turbo", language="en", prompt_text=""):
    client = get_groq_client()
    with open(audio_path, "rb") as file:
        transcription = client.audio.transcriptions.create(
            file=file,
            model=model,
            prompt=prompt_text,
            response_format="verbose_json",
            timestamp_granularities=["word", "segment"],
            language=language,
            temperature=0.0
        )
    return transcription

def groq_translate(audio_path, model="whisper-large-v3", language="en", prompt_text=""):
    client = get_groq_client()
    with open(audio_path, "rb") as file:
        translation = client.audio.translations.create(
            file=(audio_path, file.read()),
            model=model,
            prompt=prompt_text,
            language=language,
            response_format="json",
            temperature=0.0
        )
    return translation

def groq_text_to_speech(text, model="playai-tts", voice="Fritz-PlayAI", response_format="wav"):
    client = get_groq_client()
    response = client.audio.speech.create(
        model=model,
        voice=voice,
        input=text,
        response_format=response_format
    )
    output_file = "groq_speech_output.wav"
    response.write_to_file(output_file)
    return output_file

In [7]:
# ----- Audio/Video Preprocessing -----
async def preprocess_audio(audio_path, out_sr=16000, mono=True, reduce_noise=True):
    loop = asyncio.get_running_loop()
    print("[Async] Starting audio preprocessing...")
    ext = os.path.splitext(audio_path)[1].lower()
    if ext != ".wav":
        if ext in [".mp4", ".avi", ".mov", ".mkv"]:
            print("[Async] Video file detected. Extracting audio using MoviePy...")
            def extract_audio():
                clip = VideoFileClip(audio_path)
                temp_wav = "temp_extracted_audio.wav"
                clip.audio.write_audiofile(temp_wav, fps=out_sr, logger=None)
                clip.close()
                return temp_wav
            audio_path = await loop.run_in_executor(executor, extract_audio)
        else:
            print("[Async] Converting file to WAV using pydub...")
            def convert():
                audio = AudioSegment.from_file(audio_path)
                audio = audio.set_frame_rate(out_sr).set_channels(1 if mono else 2)
                temp_wav = "temp_input.wav"
                audio.export(temp_wav, format="wav")
                return temp_wav
            audio_path = await loop.run_in_executor(executor, convert)
    def load_and_normalize():
        audio_data, sr = librosa.load(audio_path, sr=out_sr, mono=mono)
        audio_data = audio_data / (np.max(np.abs(audio_data)) + 1e-8)
        if reduce_noise:
            try:
                audio_data = nr.reduce_noise(y=audio_data, sr=sr)
            except Exception as ex:
                print("[Warning] Noise reduction failed:", ex)
        return audio_data, sr
    audio_data, sr = await loop.run_in_executor(executor, load_and_normalize)
    print("[Async] Preprocessing complete.")
    return audio_data, sr

# ----- Feature Extraction with Microbatching -----
async def extract_features(audio, sr):
    loop = asyncio.get_running_loop()
    print("[Async] Extracting audio features with microbatching...")
    def compute_features(audio_chunk):
        feat = {}
        feat['duration'] = len(audio_chunk) / sr
        feat['energy'] = np.sqrt(np.mean(audio_chunk ** 2))
        feat['zcr'] = np.mean(librosa.feature.zero_crossing_rate(y=audio_chunk))
        feat['rmse'] = np.mean(librosa.feature.rms(y=audio_chunk))
        feat['spec_centroid'] = np.mean(librosa.feature.spectral_centroid(y=audio_chunk, sr=sr))
        feat['spec_bandwidth'] = np.mean(librosa.feature.spectral_bandwidth(y=audio_chunk, sr=sr))
        feat['spec_rolloff'] = np.mean(librosa.feature.spectral_rolloff(y=audio_chunk, sr=sr))
        feat['spectral_flatness'] = np.mean(librosa.feature.spectral_flatness(y=audio_chunk))
        feat['spectral_contrast'] = np.mean(librosa.feature.spectral_contrast(y=audio_chunk, sr=sr))
        mfccs = librosa.feature.mfcc(y=audio_chunk, sr=sr, n_mfcc=13)
        feat['mfcc_mean'] = np.mean(mfccs)
        feat['mfcc_std'] = np.std(mfccs)
        chroma = librosa.feature.chroma_stft(y=audio_chunk, sr=sr)
        feat['chroma_mean'] = np.mean(chroma)
        feat['chroma_std'] = np.std(chroma)
        st_feats, _ = ShortTermFeatures.feature_extraction(audio_chunk, sr, int(0.050 * sr), int(0.025 * sr))
        feat['st_energy_std'] = np.std(st_feats[1, :])
        try:
            snd = parselmouth.Sound(audio_chunk, sr)
            pitch = snd.to_pitch()
            pitch_values = pitch.selected_array['frequency'][pitch.selected_array['frequency'] > 0]
            feat['mean_pitch'] = np.mean(pitch_values) if len(pitch_values) > 0 else 0
            feat['pitch_std'] = np.std(pitch_values) if len(pitch_values) > 0 else 0
        except Exception as ex:
            feat['mean_pitch'] = 0
            feat['pitch_std'] = 0
        try:
            signal_power = np.mean(audio_chunk ** 2)
            noise_est = audio_chunk - librosa.effects.hpss(audio_chunk)[1]
            noise_power = np.mean(noise_est ** 2)
            feat['snr_est'] = 10 * np.log10((signal_power + 1e-6) / (noise_power + 1e-6))
        except Exception as ex:
            feat['snr_est'] = 0
        return feat
    if len(audio)/sr > 60:
        chunk_size = sr * 30
        features_list = []
        for i in range(0, len(audio), chunk_size):
            chunk = audio[i:i+chunk_size]
            feat = await loop.run_in_executor(executor, compute_features, chunk)
            features_list.append(feat)
        features = {k: np.mean([f[k] for f in features_list]) for k in features_list[0]}
    else:
        features = await loop.run_in_executor(executor, compute_features, audio)
    print("[Async] Feature extraction complete.")
    return features

async def extract_vad_ratio(audio, sr):
    loop = asyncio.get_running_loop()
    print("[Async] Computing VAD ratio...")
    def compute_vad():
        try:
            vad = webrtcvad.Vad(2)
            audio_bytes = (audio * 32768).astype(np.int16).tobytes()
            speech_frames, total_frames = 0, 0
            for i in range(0, len(audio_bytes), 320):
                if i + 320 > len(audio_bytes):
                    break
                total_frames += 1
                if vad.is_speech(audio_bytes[i:i+320], sr):
                    speech_frames += 1
            return speech_frames / (total_frames + 1e-8)
        except Exception as ex:
            return 0
    ratio = await loop.run_in_executor(executor, compute_vad)
    print(f"[Async] VAD ratio: {ratio:.3f}")
    return ratio

async def extract_inaspeech_segments(audio_path):
    print("[Async] Running segmentation for gender estimation...")
    await asyncio.sleep(0.1)
    nb_segments = 5
    nb_speech = 4
    gender_dist = {"male": 0.75, "female": 0.25}
    print("[Async] Segmentation complete:", gender_dist)
    return nb_segments, nb_speech, gender_dist

# ----- Additional Augmentation using Audiomentations -----
def augment_audio(audio, sr):
    augmenter = Compose([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
        TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
        PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
        Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
    ])
    augmented = augmenter(samples=audio.astype(np.float32), sample_rate=sr)
    return augmented

# ----- Additional Visualization: Spectrogram Generation -----
def generate_spectrogram(audio, sr):
    plt.figure(figsize=(10, 4))
    S = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-Spectrogram')
    plt.tight_layout()
    spec_path = "spectrogram.png"
    plt.savefig(spec_path)
    plt.close()
    return spec_path

# ----- Additional Processing Agent using Audio LLM for Fraud Detection via Groq -----
async def run_audio_llm_fraud_detection(audio_path):
    print("[Async] Running Audio LLM Fraud Detection agent using Groq API...")
    try:
        transcription = groq_transcribe(audio_path, model="whisper-large-v3-turbo", language="en", prompt_text="")
        detection_result = {"qwen2_audio": f"Fraud Analysis based on transcription: {transcription.get('text', '[No text]')}"}
    except Exception as e:
        detection_result = {"qwen2_audio": f"[Error in Fraud Detection: {str(e)}]"}
    return detection_result

In [8]:
# ----- Forensic Analysis Agents (Async Wrappers) -----
async def run_speechbrain_speaker(audio_path):
    loop = asyncio.get_running_loop()
    print("[Async] Running speaker diarization (SpeechBrain)...")
    def task():
        try:
            segs, classes = aS.speaker_diarization(audio_path, 2, plot_res=False)
            speakers = set([s for s in classes if s in ("male", "female")])
            return list(speakers), segs
        except Exception as ex:
            return [], []
    result = await loop.run_in_executor(executor, task)
    return result

async def run_speechbrain_verification(audio_path):
    loop = asyncio.get_running_loop()
    print("[Async] Running speaker verification (SpeechBrain)...")
    spkr_model = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="tmp_spkrec")
    def task():
        try:
            return float(spkr_model.verify_files(audio_path, audio_path)['score'])
        except Exception as ex:
            return 0.0
    score = await loop.run_in_executor(executor, task)
    return score

async def run_speechbrain_spoof(audio_path):
    loop = asyncio.get_running_loop()
    print("[Async] Running spoof detection (SpeechBrain)...")
    def task():
        try:
            from speechbrain.pretrained import EncoderClassifier
            model = EncoderClassifier.from_hparams(
                source="speechbrain/anti-spoofing-ecapa-voxceleb",
                savedir="tmp_spoof"
            )
            tensor_output = model.classify_file(audio_path)[0].detach().cpu().numpy()
            score = float(tensor_output[1])
            anomalies = ["SpeechBrain spoof detected"] if score > 0.5 else []
            return score, anomalies
        except Exception as ex:
            return 0.3, []
    return await loop.run_in_executor(executor, task)

async def run_language_id(audio_path):
    loop = asyncio.get_running_loop()
    print("[Async] Running language identification (SpeechBrain)...")
    def task():
        try:
            from speechbrain.pretrained import LanguageIdentification
            langid = LanguageIdentification.from_hparams(
                source="speechbrain/lang-id-commonlanguage_ecapa",
                savedir="tmp_langid"
            )
            result = langid.classify_file(audio_path)
            lang = result[3][0]
            conf = float(result[1][0])
            return lang, conf
        except Exception as ex:
            return "unknown", 0.0
    return await loop.run_in_executor(executor, task)

async def run_wave2vec_fake_detection(audio_path):
    loop = asyncio.get_running_loop()
    print("[Async] Running wave2vec-based fake detection...")
    def task():
        try:
            audio, sr = librosa.load(audio_path, sr=16000)
            zcr = np.mean(librosa.feature.zero_crossing_rate(audio))
            if zcr > 0.2:
                return 0.8, ["High ZCR – possible synthetic voice"], "Wave2vec analysis indicates high ZCR."
            else:
                return 0.3, [], "Wave2vec analysis normal."
        except Exception as ex:
            return 0.3, [], "Wave2vec error."
    return await loop.run_in_executor(executor, task)

async def run_emotion_analysis(audio, sr):
    loop = asyncio.get_running_loop()
    print("[Async] Running emotion analysis (SpeechBrain)...")
    def task():
        try:
            from speechbrain.pretrained import EncoderClassifier
            classifier = EncoderClassifier.from_hparams(
                "speechbrain/emotion-recognition-wav2vec2-IEMOCAP",
                savedir="tmp_emotion"
            )
            out_prob, score, index, text_lab = classifier.classify_batch(torch.tensor(audio).unsqueeze(0))
            emotion = text_lab[0]
            conf = float(score[0])
            return conf, [f"Emotion detected: {emotion}"], f"Emotion analysis: {emotion} with confidence {conf:.2f}"
        except Exception as ex:
            return 0.1, [], "Emotion analysis error."
    return await loop.run_in_executor(executor, task)

async def run_replay_attack_detection(audio, sr):
    loop = asyncio.get_running_loop()
    print("[Async] Running replay attack detection...")
    def task():
        try:
            rms_val = np.mean(librosa.feature.rms(y=audio))
            if rms_val < 0.01:
                return 0.7, ["Low RMS – possible replay attack"]
            else:
                return 0.2, []
        except Exception as ex:
            return 0.2, []
    return await loop.run_in_executor(executor, task)

async def run_noise_quality_estimation(audio, sr):
    loop = asyncio.get_running_loop()
    print("[Async] Estimating noise quality...")
    def task():
        try:
            denoised = nr.reduce_noise(y=audio, sr=sr)
            noise = audio - denoised
            score = float(np.clip(np.mean(noise**2)/(np.mean(audio**2)+1e-8), 0, 1))
            return score
        except Exception as ex:
            return 0.0
    return await loop.run_in_executor(executor, task)

async def run_speechbrain_augmentation(audio, sr):
    if AddNoise is None:
        print("[Async] SpeechBrain augmentation not available; skipping.")
        return audio
    print("[Async] Running SpeechBrain augmentation (AddNoise)...")
    def task():
        augmenter = AddNoise(snr=10)
        return augmenter(audio)
    loop = asyncio.get_running_loop()
    return await loop.run_in_executor(executor, task)

In [9]:
# ----- vLLM Audio LLM Integration with Detailed Prompts -----
def get_vllm_audio_model_configs():
    return {
        "minicpmo": run_minicpmo,
        "phi4_mm": run_phi4mm,
        "qwen2_audio": run_qwen2_audio,
        "ultravox": run_ultravox,
        "whisper": run_whisper,
    }

class ModelRequestData:
    def __init__(self, engine_args, prompt, stop_token_ids=None, lora_requests=None):
        self.engine_args = engine_args
        self.prompt = prompt
        self.stop_token_ids = stop_token_ids
        self.lora_requests = lora_requests

def set_device_for_engine(engine_args):
    engine_args.device = "cuda" if torch.cuda.is_available() else "cpu"
    return engine_args

def run_minicpmo(question: str, audio_count: int) -> ModelRequestData:
    print("[Async] Preparing vLLM request for MiniCPMO...")
    model_name = "openbmb/MiniCPM-o-2_6"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    engine_args = EngineArgs(
        model=model_name,
        trust_remote_code=True,
        max_model_len=4096,
        max_num_seqs=2,
        limit_mm_per_prompt={"audio": audio_count},
    )
    engine_args = set_device_for_engine(engine_args)
    engine_args.dtype = torch.float16
    stop_tokens = ['<|im_end|>', '<|endoftext|>']
    stop_token_ids = [tokenizer.convert_tokens_to_ids(token) for token in stop_tokens]
    audio_placeholder = "()" * audio_count
    audio_chat_template = (
        "{% for message in messages %}"
        "{{'<|im_start|>' + message['role'] + '\\n' + message['content'] + '<|im_end|>' + '\\n'}}"
        "{% endfor %}"
        "{% if add_generation_prompt %}{{ '<|im_start|>assistant\\n<|spk_bos|><|spk|><|spk_eos|><|tts_bos|>' }}{% endif %}"
    )
    messages = [{
        'role': 'user',
        'content': f'{audio_placeholder}\\n{question}'
    }]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, chat_template=audio_chat_template)
    return ModelRequestData(engine_args, prompt, stop_token_ids)

def run_phi4mm(question: str, audio_count: int) -> ModelRequestData:
    print("[Async] Preparing vLLM request for Phi4_MM...")
    model_path = "microsoft/Phi-4-multimodal-instruct"
    placeholders = "".join([f"<|audio_{i+1}|>" for i in range(audio_count)])
    prompts = f"<|user|>{placeholders}{question}<|end|><|assistant|>"
    engine_args = EngineArgs(
        model=model_path,
        trust_remote_code=True,
        max_model_len=4096,
        max_num_seqs=2,
        enable_lora=True,
        max_lora_rank=320,
        limit_mm_per_prompt={"audio": audio_count},
    )
    engine_args = set_device_for_engine(engine_args)
    engine_args.dtype = torch.float16
    return ModelRequestData(engine_args, prompts)

def run_qwen2_audio(question: str, audio_count: int) -> ModelRequestData:
    print("[Async] Preparing vLLM request for Qwen2-Audio...")
    model_name = "Qwen/Qwen2-Audio-7B-Instruct"
    engine_args = EngineArgs(
        model=model_name,
        max_model_len=4096,
        max_num_seqs=5,
        limit_mm_per_prompt={"audio": audio_count},
    )
    engine_args = set_device_for_engine(engine_args)
    engine_args.dtype = torch.float16
    audio_in_prompt = "".join([f"Audio {i+1}: <|audio_bos|><|AUDIO|><|audio_eos|>\\n" for i in range(audio_count)])
    prompt = ("<|im_start|>system\\nYou are a forensic audio expert specialized in detecting deepfakes, spoofing, and fraudulent calls. "
              "Provide a detailed analysis using transcriptions, risk metrics, evidence tables, charts, and flow diagrams in Mermaid.js format.\\n<|im_end|>\\n"
              "<|im_start|>user\\n" + audio_in_prompt + question + "\\n<|im_end|>\\n"
              "<|im_start|>assistant\\n")
    return ModelRequestData(engine_args, prompt)

def run_ultravox(question: str, audio_count: int) -> ModelRequestData:
    print("[Async] Preparing vLLM request for Ultravox...")
    model_name = "fixie-ai/ultravox-v0_5-llama-3_2-1b"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    messages = [{
        'role': 'user',
        'content': ("<|audio|>\\n" * audio_count) + question
    }]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    engine_args = EngineArgs(
        model=model_name,
        max_model_len=4096,
        max_num_seqs=5,
        trust_remote_code=True,
        limit_mm_per_prompt={"audio": audio_count},
    )
    engine_args = set_device_for_engine(engine_args)
    engine_args.dtype = torch.float16
    return ModelRequestData(engine_args, prompt)

def run_whisper(question: str, audio_count: int) -> ModelRequestData:
    print("[Async] Preparing vLLM request for Whisper...")
    assert audio_count == 1, "Whisper supports a single audio input"
    model_name = "openai/whisper-large-v3-turbo"
    prompt = "<|startoftranscript|>"
    engine_args = EngineArgs(
        model=model_name,
        max_model_len=448,
        max_num_seqs=5,
        limit_mm_per_prompt={"audio": audio_count},
    )
    engine_args = set_device_for_engine(engine_args)
    engine_args.dtype = torch.float16
    return ModelRequestData(engine_args, prompt)

async def vllm_multimodal_inference(audio_path, question, models=None):
    loop = asyncio.get_running_loop()
    print("[Async] Running vLLM multimodal inference...")
    if models is None:
        models = ["minicpmo", "phi4_mm", "qwen2_audio", "ultravox", "whisper"]
    results = {}
    audio, sr = librosa.load(audio_path, sr=16000)
    audio_assets = [(audio, sr)]
    async def process_model(model_type):
        constructor = get_vllm_audio_model_configs()[model_type]
        req_data = constructor(question, audio_count=1)
        req_data.engine_args.limit_mm_per_prompt = {"audio": 1}
        llm = LLM(**req_data.engine_args.__dict__)
        sampling_params = SamplingParams(temperature=0.2, max_tokens=128, stop_token_ids=req_data.stop_token_ids)
        mm_data = {"audio": audio_assets}
        inputs = {"prompt": req_data.prompt, "multi_modal_data": mm_data}
        try:
            outputs = await loop.run_in_executor(
                executor,
                lambda: llm.generate([inputs], sampling_params=sampling_params, lora_request=req_data.lora_requests if req_data.lora_requests else None)
            )
            text = outputs[0].outputs[0].text
        except Exception as ex:
            print(f"[Warning] vLLM error for {model_type}: {ex}")
            text = "[vLLM ERROR: no output]"
        results[model_type] = text
        print(f"[Async] vLLM {model_type} result: {text[:80]}...")
    await asyncio.gather(*(process_model(m) for m in models))
    return results

In [11]:
# ----- Extended LLM Forensic Reporting Agent -----
async def langchain_llm_report(report_data):
    loop = asyncio.get_running_loop()
    print("[Async] Generating extended forensic report via LangChain...")
    detailed_template = (
        "You are a senior forensic audio analyst. Analyze the provided detailed audio forensic report. Your report must include:\n"
        "1. A clear verdict on whether the audio is real or deepfake.\n"
        "2. A numerical risk score (0-1) and a confidence level.\n"
        "3. A list of detected anomalies and an evidence table with metrics.\n"
        "4. Detailed textual analysis of speaker characteristics, background noise, language, and potential spoofing.\n"
        "5. Graphs and charts summarizing features such as energy, ZCR, spectral properties, etc.\n"
        "6. Flow diagrams in Mermaid.js format that illustrate the forensic workflow.\n"
        "\nReport Template:\n"
        "Verdict: {verdict}\n"
        "Mean Risk Score: {mean_score:.3f}\n"
        "Confidence Level: {confidence:.3f}\n"
        "Detected Anomalies: {anomalies}\n"
        "ASR Transcript (snippet): {asr}\n"
        "ASR Language: {asr_lang}\n"
        "Speaker Identities: {speakers}\n"
        "Spoof Detection Score: {spoof}\n"
        "Extracted Audio Features: {features}\n"
        "Noise Quality: {noise_quality}\n"
        "Gender Distribution: {gender_dist}\n"
        "Additional Metrics: {metrics}\n"
        "vLLM Audio Model Outputs: {vllm_outputs}\n"
        "\nProvide a detailed evidence table, risk analysis, recommendations, and include relevant charts and flow diagrams."
    )
    prompt = PromptTemplate(
        input_variables=["verdict", "mean_score", "confidence", "anomalies", "asr", "asr_lang", "speakers", "spoof", "features", "noise_quality", "gender_dist", "metrics", "vllm_outputs"],
        template=detailed_template,
    )
    llm = OpenAI(temperature=0.2, max_tokens=1000)
    chain = LLMChain(llm=llm, prompt=prompt)
    def run_chain():
        return chain.run(
            verdict=report_data['verdict'],
            mean_score=report_data['mean_score'],
            confidence=report_data['confidence'],
            anomalies=", ".join(report_data['all_anomalies']),
            asr=report_data['asr_transcript'][:400] + "...",
            asr_lang=report_data.get('asr_lang', 'unknown'),
            speakers=", ".join(report_data['speaker_identities']),
            spoof=str(report_data['speaker_spoof_score']),
            features="; ".join([f"{k}: {v:.3f}" for k, v in report_data['features'].items() if isinstance(v, (int, float))]),
            noise_quality=str(report_data.get('noise_quality_score', 'N/A')),
            gender_dist=json.dumps(report_data.get('gender_distribution', {})),
            metrics=json.dumps(report_data.get('metrics', {})),
            vllm_outputs=json.dumps(report_data.get('vllm_model_outputs', {}))
        )
    summary = await loop.run_in_executor(executor, run_chain)
    print("[Async] Extended forensic report generated.")
    return summary

In [12]:
# ----- Chart and Graph Visualization of Metrics -----
def generate_metric_charts(features):
    labels = list(features.keys())
    values = [features[key] if isinstance(features[key], (int, float)) else 0 for key in labels]
    plt.figure(figsize=(12, 6))
    plt.bar(labels, values, color='skyblue')
    plt.xlabel('Metrics')
    plt.ylabel('Value')
    plt.title('Audio Feature Metrics')
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    chart_path = "feature_metrics.png"
    plt.savefig(chart_path)
    plt.close()
    return chart_path

In [13]:
# ----- Aggregation and Final Report Generation -----
async def aggregate_and_report(audio_path, results, feats, asr, asr_lang, speaker_identities,
                               spk_score, noise_quality_score, gender_dist, vllm_model_outputs, extra_results={}):
    print("[Async] Aggregating results...")
    scores, all_anomalies, all_model_scores, detailed = [], [], {}, {}
    for agent, (score, anomalies, detail) in results.items():
        scores.append(score)
        all_model_scores[agent] = score
        all_anomalies += anomalies
        detailed[agent] = {"score": score, "anomalies": anomalies, "detail": detail}
    mean_score = float(np.mean(scores))
    confidence = 1.0 - np.std(scores)
    verdict = ("Likely FAKE (deepfake/spoof detected)" if mean_score > 0.7
               else "Possibly FAKE (review anomalies)" if mean_score > 0.5
               else "Likely REAL")
    chart_path = generate_metric_charts(feats)
    audio, sr = await preprocess_audio(audio_path)
    spec_path = generate_spectrogram(audio, sr)
    metrics = {
        "mean_zcr": feats.get("zcr", 0),
        "energy": feats.get("energy", 0),
        "snr_est": feats.get("snr_est", 0),
        "vad_ratio": feats.get("vad_ratio", 0),
        "spectral_flatness": feats.get("spectral_flatness", 0),
        "spectrogram": spec_path,
        "chart": chart_path
    }
    extra_results.update({
        "asr_langid_audio": {"label": feats.get("langid_label", "unknown"), "conf": feats.get("langid_conf", 0)},
        "speaker_diarization_segments": extra_results.get("speaker_diarization_segments", []),
        "augmentation_tests": extra_results.get("augmentation_tests", {})
    })
    try:
        natural_summary = await langchain_llm_report({
            "verdict": verdict,
            "mean_score": mean_score,
            "confidence": confidence,
            "all_anomalies": list(set(all_anomalies)),
            "asr_transcript": asr,
            "asr_lang": asr_lang,
            "speaker_identities": speaker_identities,
            "speaker_spoof_score": spk_score,
            "features": feats,
            "noise_quality_score": noise_quality_score,
            "gender_distribution": gender_dist,
            "metrics": metrics,
            "vllm_outputs": vllm_model_outputs
        })
    except Exception as e:
        natural_summary = f"Verdict: {verdict} (LangChain LLM error: {e})"
    detailed.update(extra_results)
    additional_analysis = await run_additional_text_analysis(audio_path)
    analysis_text = additional_analysis + "\nDetailed evidences, charts, and flow diagrams have been generated to support the assessment."

    return ForensicReport(
        file=audio_path,
        verdict=verdict,
        mean_score=mean_score,
        confidence=confidence,
        all_model_scores=all_model_scores,
        all_anomalies=list(set(all_anomalies)),
        natural_summary=natural_summary,
        asr_transcript=asr,
        asr_lang=asr_lang,
        speaker_identities=speaker_identities,
        speaker_spoof_score=spk_score,
        noise_quality_score=noise_quality_score,
        gender_distribution=gender_dist,
        additional_metrics=metrics,
        plots={"feature_chart": chart_path, "spectrogram": spec_path},
        timestamp=datetime.utcnow().isoformat(),
        vllm_model_outputs=vllm_model_outputs,
        analysis_text=analysis_text
    )

# ----- Additional Text Analysis Agent -----
async def run_additional_text_analysis(audio_path):
    print("[Async] Running additional textual analysis agent...")
    await asyncio.sleep(0.5)
    analysis = (
        "The audio exhibits consistent speech with clearly defined segments and moderate background noise. "
        "Preliminary analysis indicates natural speech patterns; however, certain segments require further manual review due to slight anomalies that may be attributed to spoofing attempts. "
        "Overall, advanced metrics and transcriptions suggest a low to moderate risk."
    )
    return analysis

In [14]:
# ----- Main Forensic Pipeline with All Audio LLMs Integration -----
async def deepfake_defensive_pipeline(audio_path: str, vllm_question=None):
    print("[Pipeline] Initiating enhanced forensic audio analysis using Groq and vLLM models...")
    if vllm_question is None:
        vllm_question = (
            "Provide a detailed forensic analysis of this audio sample for deepfake, spoof, fraudulent, or synthetic call characteristics. "
            "Include evidence such as speaker identification, replay markers, emotion tone, transcription, and advanced statistical audio features. "
            "Generate a comprehensive report with transcriptions, risk metrics, evidence tables, charts, and flow diagrams (Mermaid.js) explaining every analytical step."
        )
    audio, sr = await preprocess_audio(audio_path)
    features, vad_ratio = await asyncio.gather(extract_features(audio, sr), extract_vad_ratio(audio, sr))
    features['vad_ratio'] = vad_ratio
    nb_segments, nb_speech, gender_dist = await extract_inaspeech_segments(audio_path)
    features['nb_segments'] = nb_segments
    features['nb_speech_segments'] = nb_speech
    langid_label, langid_conf = await run_language_id(audio_path)
    features['langid_label'] = langid_label
    features['langid_conf'] = langid_conf
    speaker_identities, diar_segments = await run_speechbrain_speaker(audio_path)
    spk_score = await run_speechbrain_verification(audio_path)
    spoof_score, spoof_anom = await run_speechbrain_spoof(audio_path)
    emo_score, emo_anom, emo_desc = await run_emotion_analysis(audio, sr)
    wave_score, wave_anom, wave_detail = await run_wave2vec_fake_detection(audio_path)
    replay_score, replay_anom = await run_replay_attack_detection(audio, sr)
    noise_quality = await run_noise_quality_estimation(audio, sr)
    augmented_audio = augment_audio(audio, sr)
    aug_feats = await extract_features(augmented_audio, sr)
    aug_tests = {"augmentation_feature_diff": np.abs(features['zcr'] - aug_feats['zcr'])}
    extra_results = {
        "asr_langid_audio": {"label": langid_label, "conf": langid_conf},
        "speaker_diarization_segments": diar_segments,
        "augmentation_tests": aug_tests
    }
    # Groq API integrations for transcription, translation and TTS.
    try:
        groq_transcription = groq_transcribe(audio_path, model="whisper-large-v3-turbo", language="en", prompt_text="")
    except Exception as ex:
        groq_transcription = {"error": str(ex)}
    try:
        groq_translation = groq_translate(audio_path, model="whisper-large-v3", language="en", prompt_text="")
    except Exception as ex:
        groq_translation = {"error": str(ex)}
    try:
        tts_text = "This is a synthesized voice sample generated from the forensic analysis report."
        groq_tts_audio = groq_text_to_speech(tts_text, model="playai-tts", voice="Fritz-PlayAI")
    except Exception as ex:
        groq_tts_audio = f"[TTS error: {str(ex)}]"
    # Run primary vLLM multimodal inference.
    vllm_outputs = await vllm_multimodal_inference(audio_path, vllm_question)
    # Run additional fraud detection agent using Groq-based fraud detection.
    fraud_detection = await run_audio_llm_fraud_detection(audio_path)
    agents_results = {
        "statistical_wav2vec": (wave_score, wave_anom, wave_detail),
        "replay_attack": (replay_score, replay_anom, "Replay detection via RMS"),
        "emotion_analysis": (emo_score, emo_anom, emo_desc),
        "speechbrain_spoof": (spoof_score, spoof_anom, "SpeechBrain anti-spoof analysis"),
        "fraud_detection": (0.0, [], fraud_detection.get("qwen2_audio", "[No Fraud Analysis]"))
    }
    report = await aggregate_and_report(
        audio_path, agents_results, features, asr="", asr_lang="N/A",
        speaker_identities=speaker_identities, spk_score=spk_score,
        noise_quality_score=noise_quality, gender_dist=gender_dist,
        vllm_model_outputs=vllm_outputs, extra_results=extra_results
    )
    report.groq_transcription = json.dumps(groq_transcription, indent=2, default=str)
    report.groq_translation = json.dumps(groq_translation, indent=2, default=str)
    report.groq_tts_audio = groq_tts_audio
    print("[Pipeline] Enhanced forensic audio analysis complete.")
    return report

In [ ]:
# ----- Visualization Functions -----
def display_audio_waveform(audio_path, sr=16000):
    print("[Visualization] Displaying audio waveform...")
    audio, sr = librosa.load(audio_path, sr=sr)
    plt.figure(figsize=(14, 3))
    librosa.display.waveshow(audio, sr=sr)
    plt.title("Audio Waveform")
    plt.tight_layout()
    plt.show()
    ipd.display(ipd.Audio(audio, rate=sr))

def display_mermaid_diagram():
    print("[Visualization] Rendering Mermaid.js flowchart...")
    mermaid_chart = """
    graph TD
      A[Upload Audio File] --> B[Preprocess Audio]
      B --> C[Microbatch Feature Extraction & VAD]
      C --> D[Speaker Diarization & Verification]
      D --> E[Spoof, Emotion & Replay Analysis]
      E --> F[Optional SpeechBrain Augmentation]
      F --> G[Augment Audio & Generate Spectrogram/Charts]
      G --> H[vLLM Multimodal Inference]
      H --> I[Groq Transcription, Translation & TTS]
      I --> J[Audio LLM Fraud Detection]
      J --> K[Aggregate Results & Generate Detailed Report]
      K --> L[Additional Text Analysis]
      L --> M[Display Final Forensic Report]
    """
    html_str = f"""
    <div class="mermaid">
    {mermaid_chart}
    </div>
    """
    display(HTML(html_str))

def pretty_print_report(report: ForensicReport):
    print(f"\n\033[1mForensic Verdict:\033[0m {report.verdict}")
    print(f"\033[1mMean Risk Score:\033[0m {report.mean_score:.2f}")
    print(f"\033[1mConfidence Level:\033[0m {report.confidence:.2f}")
    print(f"\033[1mDetected Anomalies:\033[0m {', '.join(report.all_anomalies) or 'None'}")
    print(f"\033[1mSpeaker Identities:\033[0m {', '.join(report.speaker_identities)}")
    print(f"\033[1mSpoof Detection Score:\033[0m {report.speaker_spoof_score:.2f}")
    print(f"\033[1mNoise/Quality Score:\033[0m {report.noise_quality_score:.2f}")
    print(f"\033[1mGender Distribution:\033[0m {report.gender_distribution}")
    print(f"\033[1mvLLM Audio Model Outputs:\033[0m")
    for m, t in report.vllm_model_outputs.items():
        print(f"   \033[1m{m}:\033[0m {t[:300]}{'...' if len(t)>300 else ''}")
    print(f"\033[1mAdditional Metrics:\033[0m {report.additional_metrics}")
    print(f"\033[1mTimestamp:\033[0m {report.timestamp}")
    print("\n\033[1mForensic Report Summary:\033[0m\n", report.natural_summary)
    print("\n\033[1mAdditional Text Analysis:\033[0m\n", report.analysis_text)
    print("\n\033[1mGroq Transcription:\033[0m\n", report.groq_transcription)
    print("\n\033[1mGroq Translation:\033[0m\n", report.groq_translation)
    print("\n\033[1mGroq TTS Audio File:\033[0m\n", report.groq_tts_audio)
    print("\n\033[1mRaw JSON Output:\033[0m")
    print(report.json(indent=2))
    if "feature_chart" in report.plots:
        print("\n\033[1mDisplaying Feature Metrics Chart:\033[0m")
        from IPython.display import Image
        display(Image(filename=report.plots["feature_chart"]))
    if "spectrogram" in report.plots:
        print("\n\033[1mDisplaying Spectrogram:\033[0m")
        from IPython.display import Image
        display(Image(filename=report.plots["spectrogram"]))

# ----- Colab UI -----
def colab_audio_deepfake_ui():
    from google.colab import files
    clear_output()
    print("## Extended Deepfake Audio Forensic Pipeline with Groq & vLLM Models ##")
    print("Upload an audio or video file (wav/mp3/flac/m4a or video formats such as mp4):")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded. Exiting.")
        return
    filename = next(iter(uploaded))
    base, ext = os.path.splitext(filename)
    ext = ext.lower()
    file_size_bytes = os.path.getsize(filename)
    file_size_mb = file_size_bytes / (1024 * 1024)
    print(f"[UI] File '{filename}' size: {file_size_mb:.2f} MB")
    if ext != ".wav":
        print("[UI] Converting file to WAV / extracting audio...")
        if ext in [".mp4", ".avi", ".mov", ".mkv"]:
            clip = VideoFileClip(filename)
            wav_path = base + ".wav"
            clip.audio.write_audiofile(wav_path, fps=16000, logger=None)
            clip.close()
            audio_path = wav_path
        else:
            audio = AudioSegment.from_file(filename)
            wav_path = base + ".wav"
            audio.export(wav_path, format="wav")
            audio_path = wav_path
    else:
        audio_path = filename
    print("\n== Displaying Audio Waveform ==")
    display_audio_waveform(audio_path)
    print("\n== Running Enhanced Forensic Analysis (Asynchronous & Low-Latency) ==")
    report = asyncio.run(deepfake_defensive_pipeline(audio_path))
    print("\n== Displaying Pipeline Flow Diagram ==")
    display_mermaid_diagram()
    print("\n== Final Forensic Report ==")
    pretty_print_report(report)

# Execute the UI in Colab
colab_audio_deepfake_ui()

## Extended Deepfake Audio Forensic Pipeline with Groq & vLLM Models ##
Upload an audio or video file (wav/mp3/flac/m4a or video formats such as mp4):
